In [ ]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ['KERAS_BACKEND']= 'jax'
import keras_nlp
import keras

In [ ]:
keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  2664k      0  0:00:30  0:00:30 --:--:-- 4307k


In [ ]:
BATCH_SIZE= 16
imdb_train=keras.utils.text_dataset_from_directory(
    '/content/aclImdb/train',
    batch_size=BATCH_SIZE
)
imdb_test=keras.utils.text_dataset_from_directory(
    '/content/aclImdb/test',
    batch_size=BATCH_SIZE,
)

print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'The movie is wonderful. It shows the man\'s work for the wilderness and a natural understanding of the harmony of nature, without being an "extreme" naturalist. I definitely plan to look for the book. This is a rare treasure!<br /><br />'>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [ ]:
classifier=keras_nlp.models.BertClassifier.from_preset('bert_tiny_en_uncased_sst2')
classifier.predict(['i love modular workflows in kelas-nlp!'])

100%|██████████| 454/454 [00:00<00:00, 511kB/s]


100%|██████████| 2.04k/2.04k [00:00<00:00, 2.08MB/s]


100%|██████████| 226k/226k [00:00<00:00, 290kB/s]


100%|██████████| 50.3M/50.3M [00:04<00:00, 11.2MB/s]
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 84 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


100%|██████████| 16.8M/16.8M [00:02<00:00, 6.52MB/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[-0.7373,  0.8506]], dtype=float16)

In [ ]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.4551 - sparse_categorical_accuracy: 0.7898


[0.45500364899635315, 0.7903199791908264]

In [ ]:
classifier=keras_nlp.models.BertClassifier.from_preset('bert_tiny_en_uncased',
                                                       num_classes=2)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1
)

100%|██████████| 507/507 [00:00<00:00, 1.00MB/s]


100%|██████████| 16.8M/16.8M [00:02<00:00, 6.75MB/s]


100%|██████████| 547/547 [00:00<00:00, 456kB/s]


100%|██████████| 226k/226k [00:00<00:00, 292kB/s]


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 64s 35ms/step - loss: 0.5232 - sparse_categorical_accuracy: 0.7250 - val_loss: 0.3033 - val_sparse_categorical_accuracy: 0.8718


In [ ]:
import tensorflow as tf

preprocessor= keras_nlp.models.BertPreprocessor.from_preset(
    'bert_tiny_en_uncased',
    sequence_length=512
)

imdb_train_cached=(
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached=(
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier =keras_nlp.models.BertClassifier.from_preset(
    'bert_tiny_en_uncased', preprocessor=None, num_classes=2
)

classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3
)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 61s 36ms/step - loss: 0.5281 - sparse_categorical_accuracy: 0.7143 - val_loss: 0.3047 - val_sparse_categorical_accuracy: 0.8704
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 18ms/step - loss: 0.2792 - sparse_categorical_accuracy: 0.8856 - val_loss: 0.2858 - val_sparse_categorical_accuracy: 0.8788
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 48s 31ms/step - loss: 0.2087 - sparse_categorical_accuracy: 0.9189 - val_loss: 0.3026 - val_sparse_categorical_accuracy: 0.8786


In [ ]:
tokenizer=keras_nlp.models.BertTokenizer.from_preset('bert_tiny_en_uncased')
tokenizer(['I love modular workflows!', 'Libraries over frameworks!'])

packer= keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    sequence_length=64,
)

def preprocessor(x,y):
  token_ids, segment_ids=packer(tokenizer(x))
  x={
      'token_ids':token_ids,
      'segment_ids':segment_ids,
      'padding_mask':token_ids != 0,
  }
  return x, y

imdb_train_preprocessed= imdb_train.map(preprocessor, tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
imdb_test_preprocessed= imdb_test.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

print(imdb_train_preprocessed.unbatch().take(1).get_single_element())

({'token_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  2009,  1005,  1055,  2025,  1037,  6659,  3185,  1010,
        2428,  1010,  1998,  9465,  1998, 26679,  9834,  2024,  2327,
        1011, 18624,  5889,  1012,  2582,  1010,  2027,  2079,  2019,
       11701,  3105,  2007,  1996,  2200,  5410,  5896,  1012,  1996,
       17363,  2003, 16299,  1998,  1996,  5436,  2038,  2070,  5875,
       21438,  1012,  2582,  1010,  1045,  8529, 13375,  5794,  2094,
        2339,  2122,  5889,  1998,  1996,  3626,  2081,  1996,  2143,
         102], dtype=int32)>, 'segment_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(64,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  T

In [ ]:
preprocessor= keras_nlp.models.BertPreprocessor.from_preset('bert_tiny_en_uncased')
backbone=keras_nlp.models.BertBackbone.from_preset('bert_tiny_en_uncased')
imdb_train_preprocessed = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

imdb_test_preprocessed = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

backbone.trainable= False
inputs=backbone.input
sequence= backbone(inputs)['sequence_output']

for _ in range(2):
  sequence= keras_nlp.layers.TransformerEncoder(
      num_heads=2,
      intermediate_dim=512,
      dropout=0.1
  )(sequence)

  outputs=keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])

  model=keras.Model(inputs,outputs)

  model.compile(
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=keras.optimizers.AdamW(5e-5),
      metrics=[keras.metrics.SparseCategoricalAccuracy()],
      jit_compile=True,
  )

  model.summary()
  model.fit(imdb_train_preprocessed,
            validation_data=imdb_test_preprocessed,
            epochs=3
            )

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ padding_mask[0][0],    │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 128)      │        198,272 │ bert_backbone[0][1]    │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_5 (GetItem)      │ (None, 128)            │              0 │ transformer_encoder[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2)              │            258 │ get_item_5[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,584,450 (17.49 MB)

 Trainable params: 198,530 (775.51 KB)

 Non-trainable params: 4,385,920 (16.73 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ padding_mask[0][0],    │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 128)      │        198,272 │ bert_backbone[0][1]    │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_1     │ (None, None, 128)      │        198,272 │ transformer_encoder[0… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_6 (GetItem)      │ (None, 128)            │              0 │ transformer_encoder_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 2)              │            258 │ get_item_6[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,782,722 (18.24 MB)

 Trainable params: 396,802 (1.51 MB)

 Non-trainable params: 4,385,920 (16.73 MB)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 63s 36ms/step - loss: 0.5162 - sparse_categorical_accuracy: 0.7465 - val_loss: 0.4212 - val_sparse_categorical_accuracy: 0.8074
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - loss: 0.4482 - sparse_categorical_accuracy: 0.7902 - val_loss: 0.4053 - val_sparse_categorical_accuracy: 0.8176
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - loss: 0.4371 - sparse_categorical_accuracy: 0.7975 - val_loss: 0.3986 - val_sparse_categorical_accuracy: 0.8210


In [ ]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    'bert_tiny_en_uncased',
    sequence_length=256,
)
packer = preprocessor.packer
tokenizer = preprocessor.tokenizer

masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=0.25,
    mask_selection_length=64,
    mask_token_id=tokenizer.token_to_id('[MASK]'),
    unselectable_token_ids=[
        tokenizer.token_to_id(x) for x in ['[CLS]','[PAD]','[SEP]']
    ],
)

def preprocess(inputs, outputs):
  inputs=preprocessor(inputs)
  masked_inputs = masker(inputs['token_ids'])

  features = {
      'token_ids':masked_inputs['token_ids'],
      'segment_ids':inputs['segment_ids'],
      'padding_mask':inputs['padding_mask'],
      'mask_positions':masked_inputs['mask_positions']
  }

  labels=masked_inputs['mask_ids']
  weights=masked_inputs['mask_weights']
  return features, labels, weights

pretrain_ds= imdb_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
pretrain_val_ds=imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print(pretrain_ds.unbatch().take(1).get_single_element())

({'token_ids': <tf.Tensor: shape=(256,), dtype=int32, numpy=
array([  101,  1045,   103,  2464,   103,  3398,  4595,   103,  3152,
        1012,  1006,  2026,  5440,  2828,  1007,  2023,   103,  2003,
         103,  1996,  5409,   103,  2005,  2033,   103,  2019,   103,
        9297,  1037,  3185,  1012,  2061,  1010,  2130,  1037,   103,
        6051,  1010, 10036,  2135,  2081,  3185,  2064,  2022,   103,
        1012,  2242,  6881,   103,  5791, 10036,  2135,  2081,  1012,
        2174,   103,  2009,   103,   103,  2000,  2360, 23754,  1045,
        2145,  2123,  1005,  1056,  3305,  2054,  1996, 16894,  3496,
        1999,  1996,  2927,   103,  2000,   103,  2007,  1996,  2143,
        1012,  2242,  6881,  2038,   103,  2000,   103,   103,  3828,
        4426,  1996,  3255,   999,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  